In [1]:
import requests
from requests import Response
aqi_url = 'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate desc&format=JSON'
response:Response = requests.get(aqi_url)
if response.status_code == 200:
    print("下載成功")
else:
    print("下載失敗")


下載成功


In [5]:
#python的資料結構
data_base:dict = response.json()

In [ ]:
#json的字串格式
from pprint import pprint
pprint(response.text)
data_json:str = response.text

In [20]:
from pydantic import BaseModel,Field,field_validator

class Site(BaseModel):
    站點:str = Field(alias='sitename')
    縣市:str = Field(alias='county')
    aqi:int = Field(alias='aqi')
    pm25:float = Field(alias='pm2.5_avg')
    時間:str = Field(alias='datacreationdate')

    @field_validator('pm25',mode='before')
    @classmethod
    def space_zeor(cls,value):
        #print(type(value))
        if value == '':
            return '0.0'
        return value


class AQI(BaseModel):
    records:list[Site]




In [ ]:
AQI.model_validate(data_base)

In [ ]:
aqi = AQI.model_validate_json(data_json)
source_data = aqi.model_dump()['records']
source_data

In [25]:
import pandas as pd
df1 = pd.DataFrame(source_data)
df1

,站點,縣市,aqi,pm25,時間
0,屏東(枋山),屏東縣,31,9.2,2024-03-24 14:00
1,新北(樹林),新北市,63,20.5,2024-03-24 14:00
2,屏東(琉球),屏東縣,43,13.1,2024-03-24 14:00
3,臺南(麻豆),臺南市,47,14.5,2024-03-24 14:00
4,高雄(湖內),高雄市,31,9.0,2024-03-24 14:00
...,...,...,...,...,...
995,萬華,臺北市,70,23.2,2024-03-24 03:00
996,中山,臺北市,65,21.3,2024-03-24 03:00
997,士林,臺北市,59,18.9,2024-03-24 03:00
998,淡水,新北市,53,16.3,2024-03-24 03:00


In [29]:
#如何選取多個欄位
df2 = df1[['縣市','pm25','aqi']].copy()
df2

,縣市,pm25,aqi
0,屏東縣,9.2,31
1,新北市,20.5,63
2,屏東縣,13.1,43
3,臺南市,14.5,47
4,高雄市,9.0,31
...,...,...,...
995,臺北市,23.2,70
996,臺北市,21.3,65
997,臺北市,18.9,59
998,新北市,16.3,53


In [30]:
#選取列,透過索引(index)
#iloc[]
#loc[]
df2.iloc[0]

縣市      屏東縣
pm25    9.2
aqi      31
Name: 0, dtype: object

In [31]:
#指定多筆索引
df2.iloc[[0, 996, 999]]

,縣市,pm25,aqi
0,屏東縣,9.2,31
996,臺北市,21.3,65
999,新北市,16.6,54


In [34]:
#使用iloc[:]
df2.iloc[:10]

,縣市,pm25,aqi
0,屏東縣,9.2,31
1,新北市,20.5,63
2,屏東縣,13.1,43
3,臺南市,14.5,47
4,高雄市,9.0,31
5,彰化縣,19.6,61
6,彰化縣,17.1,55
7,新北市,15.5,51
8,雲林縣,17.0,55
9,臺東縣,9.0,29


In [36]:
df3 = df2.iloc[-10:].copy()
df3

,縣市,pm25,aqi
990,桃園市,16.0,52
991,桃園市,21.0,65
992,臺北市,25.0,74
993,臺北市,24.4,73
994,臺北市,22.7,69
995,臺北市,23.2,70
996,臺北市,21.3,65
997,臺北市,18.9,59
998,新北市,16.3,53
999,新北市,16.6,54


In [42]:
df3.aqi >= 70

990    False
991    False
992     True
993     True
994    False
995     True
996    False
997    False
998    False
999    False
Name: aqi, dtype: bool

In [43]:
#搜尋資料,使用subscript
df3[df3.aqi >= 70]

,縣市,pm25,aqi
992,臺北市,25.0,74
993,臺北市,24.4,73
995,臺北市,23.2,70
